# model L4

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from extern.AlphaForge.alphagen_generic.features import volume
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_L4, FEAT_FRACDIFF, DeepSSMContainer, LGSSMContainer

df_feat_raw = pd.read_parquet("data/feat_hard_L4.parquet")
model_deep_ssm = DeepSSMContainer()
model_lg_ssm = LGSSMContainer()

train_mask = df_feat_raw.index.to_numpy() < date_to_timestamp("2025-03-01")
train_raw = df_feat_raw.iloc[train_mask]
train_deep_ssm = model_deep_ssm.transform(df_feat_raw[FEAT_FRACDIFF].iloc[train_mask])
train_lg_ssm = model_lg_ssm.transform(df_feat_raw[FEAT_FRACDIFF].iloc[train_mask])

df_feat_l4 = pd.concat([train_deep_ssm, train_lg_ssm, train_raw], axis=1)[FEAT_L4]
print(f"{df_feat_l4.shape = }")

label = np.load("data/label_hard_L4.npy")
train_y = label[train_mask]

assert df_feat_l4.shape[0] == train_y.shape[0]

print(np.unique(train_y, return_counts=True))

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.pt
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.pt
df_feat_l4.shape = (4632, 2179)
(array([0, 1]), array([2079, 2553]))


In [5]:
import lightgbm as lgb
import optuna
from sklearn.metrics import (
    f1_score,
)

METRIC = "f1"

def eval_metric(preds, eval_dataset):
    metric_name = METRIC
    y_true = eval_dataset.get_label()
    value = f1_score(y_true, preds > 0.5, average="weighted")
    higher_better = True
    return metric_name, value, higher_better


def objective(trial):
    params = {
        "objective": "binary",
        "metric": METRIC,
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(df_feat_l4, train_y)
    # dtest = lgb.Dataset(side_features_test, side_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1500),
        feval=eval_metric,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=200, n_jobs=1)

[I 2025-09-17 16:02:15,786] A new study created in memory with name: no-name-dee89af7-940e-425c-bc10-7c38d5416ef2
[I 2025-09-17 16:02:28,591] Trial 0 finished with value: 0.8060613319232882 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 244, 'max_depth': 624, 'min_gain_to_split': 0.11371841446874122, 'min_data_in_leaf': 408, 'lambda_l1': 44.3173982523281, 'lambda_l2': 24.988043970864148, 'num_boost_round': 1336}. Best is trial 0 with value: 0.8060613319232882.
[I 2025-09-17 16:02:44,328] Trial 1 finished with value: 0.8193337196343343 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 32, 'max_depth': 896, 'min_gain_to_split': 0.8728147678519839, 'min_data_in_leaf': 140, 'lambda_l1': 13.85975459462716, 'lambda_l2': 25.357055325306163, 'num_boost_round': 689}. Best is trial 1 with value: 0.8193337196343343.
[I 2025-09-17 16:02:52,766] Trial 2 finished with value: 0.8063874677683336 and parameters:

In [7]:
print("side model long label: ")
study.best_params

side model long label: 


{'is_unbalance': False,
 'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 165,
 'max_depth': 886,
 'min_gain_to_split': 0.064492711554688,
 'min_data_in_leaf': 225,
 'lambda_l1': 1.8569025158147257,
 'lambda_l2': 38.72567699562049,
 'num_boost_round': 626}

In [9]:
params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

dtrain = lgb.Dataset(df_feat_l4, train_y)
side_model = lgb.train(params, dtrain)
# side_model_prod = lgb.train(params, lgb.Dataset(features, label))

In [10]:
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_L4, FEAT_FRACDIFF, DeepSSMContainer, LGSSMContainer

df_feat_raw = pd.read_parquet("data/feat_hard_L4.parquet")
model_deep_ssm = DeepSSMContainer()
model_lg_ssm = LGSSMContainer()

df_deep_ssm = model_deep_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_feat_l4_full = pd.concat([df_deep_ssm, df_lg_ssm, df_feat_raw], axis=1)[FEAT_L4]

label_full = np.load("data/label_hard_L4.npy")

dtrain = lgb.Dataset(df_feat_l4_full, label_full)
side_model_prod = lgb.train(params, dtrain)

Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.pt
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.pt


In [11]:
import plotly.express as px

test_features = df_feat_l4_full[df_feat_l4_full.index.to_numpy() >= date_to_timestamp("2025-03-01")]
res = side_model.predict(test_features)

fig = px.histogram(res, nbins=100)
fig.show()

In [14]:
train_features = df_feat_l4_full[df_feat_l4_full.index.to_numpy() < date_to_timestamp("2025-03-01")]
res = side_model.predict(train_features)

In [22]:
candles = np.load("data/bar_deap_v1.npy")
timestamps = candles[:, 0].astype(int)
mask = (timestamps >= train_features.index[0]) & (timestamps <= train_features.index[-1])
volume = candles[:, 5][mask]
volume.shape

(4632,)

In [23]:
df_pred = pd.DataFrame.from_dict(
    {
        "eob": pd.to_datetime(train_features.index, unit='ms'),
        "score": res,
        "volume": volume,
    }
)
df_pred.to_csv("model_l4_scores.csv", index=False)
df_pred.head()

,eob,score,volume
0,2022-08-26 12:57:00,0.077050,95252.884
1,2022-08-26 14:00:00,0.099512,88399.027
2,2022-08-26 15:16:00,0.003245,225110.927
3,2022-08-26 16:37:00,0.003687,105353.644
4,2022-08-26 23:18:00,0.011412,160951.188


In [12]:
side_model.save_model("strategies/BinanceBtcDeapV1Voting/models/model_l4.txt")
side_model_prod.save_model(
    "strategies/BinanceBtcDeapV1Voting/models/model_l4_prod.txt"
)

# meta model

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import (
    META_ALL,
    SIDE,
    get_side_model,
)

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")
print(f"{np.unique(meta_label, return_counts=True) = }")

side_model = get_side_model(False)

side_model_res = side_model.predict(df_features[SIDE])

df_features["model"] = side_model_res

meta_features = df_features[META_ALL]
print(meta_features.shape)
print(meta_label.shape)

mask = meta_features.index < date_to_timestamp("2025-01-01")
meta_features_masked = meta_features[mask]
meta_label_masked = meta_label[mask]

print(meta_features_masked.shape)
print(meta_label_masked.shape)
print(f"{np.unique(meta_label_masked, return_counts=True) = }")

meta_features.isna().sum(axis=0).sort_values(ascending=False)

np.unique(meta_label, return_counts=True) = (array([0., 1.]), array([  669, 12550]))
(13219, 2664)
(13219,)
(12115, 2664)
(12115,)
np.unique(meta_label_masked, return_counts=True) = (array([0., 1.]), array([  632, 11483]))


approximate_entropy_win128_spot    0
cwt_win1024_13_lag2                0
cwt_win256_11_dt_lag8              0
cwt_win128_8_lag4                  0
cwt_win64_12_dt                    0
                                  ..
cwt_win512_20_dt_lag11             0
cwt_win1024_11_dt_lag14            0
cwt_win128_1_dt_lag2               0
adx_7_lag3                         0
model                              0
Length: 2664, dtype: int64

In [2]:
import lightgbm as lgb
from sklearn.metrics import (
    f1_score,
    fbeta_score,
)

METRIC = "f1"


def eval_metric(preds, eval_dataset):
    metric_name = METRIC
    y_true = eval_dataset.get_label()
    value = f1_score(y_true, preds > 0.5, average="weighted")
    higher_better = True
    return metric_name, value, higher_better


params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(meta_features, meta_label)
res = lgb.cv(
    params, dtrain, num_boost_round=100, nfold=5, stratified=True, feval=eval_metric
)
res.keys()

dict_keys(['valid binary_logloss-mean', 'valid binary_logloss-stdv', 'valid f1-mean', 'valid f1-stdv'])

In [3]:
import optuna


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features, meta_label)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1500),
        stratified=True,
        feval=eval_metric,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(n_startup_trials=50),
)
study.optimize(objective, n_trials=200, n_jobs=1)

[I 2025-07-01 10:10:04,971] A new study created in memory with name: no-name-1188c647-119a-443b-b088-01c9353277eb
[I 2025-07-01 10:10:12,087] Trial 0 finished with value: 0.924743520258233 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 140, 'max_depth': 951, 'min_gain_to_split': 0.8206526039318547, 'min_data_in_leaf': 201, 'lambda_l1': 12.089838569582271, 'lambda_l2': 95.92885685399514, 'num_boost_round': 193}. Best is trial 0 with value: 0.924743520258233.
[I 2025-07-01 10:10:32,056] Trial 1 finished with value: 0.924743520258233 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 72, 'max_depth': 576, 'min_gain_to_split': 0.5274289677732332, 'min_data_in_leaf': 250, 'lambda_l1': 36.60501958746357, 'lambda_l2': 9.341901524626236, 'num_boost_round': 1043}. Best is trial 0 with value: 0.924743520258233.
[I 2025-07-01 10:10:37,441] Trial 2 finished with value: 0.9280243308865458 and parameters: {'i

In [4]:
print("meta model: ")
study.best_params

meta model: 


{'is_unbalance': True,
 'extra_trees': False,
 'boosting': 'dart',
 'num_leaves': 234,
 'max_depth': 579,
 'min_gain_to_split': 0.4783776963600167,
 'min_data_in_leaf': 254,
 'lambda_l1': 5.143714268925239,
 'lambda_l2': 21.356313125652914,
 'num_boost_round': 715}

In [5]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_masked, meta_label_masked),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [6]:
import plotly.express as px

res = model.predict(
    meta_features[meta_features.index > date_to_timestamp("2025-01-01")]
)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [7]:
model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta.txt")
prod_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta_prod.txt")